### Wednesday, December 27, 2023

Validating this still works, on container hfpt_Dec14

### Thursday, November 16, 2023

https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ

This all works!  9706Mib VRAM!

In [1]:
#!rm -rf /root/.cache/huggingface/hub

In [ ]:
# docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--Llama-2-13b-Chat-GPTQ c8324b70601d://root/.cache/huggingface/hub
# Successfully copied 7.26GB to c8324b70601d://root/.cache/huggingface/hub

In [3]:
#!mkdir /root/.cache/huggingface/hub

In [6]:
!ls /root/.cache/huggingface/hub

models--TheBloke--Llama-2-13b-Chat-GPTQ


In [5]:
# You need this to just target the 4090, othewise the 34B model does NOT load.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Remember the model name is case sensitive! Make sure it exactly matches the name of the model in the model folder!

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_name_or_path = "TheBloke/Llama-2-13b-Chat-GPTQ"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [9]:
%%time
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

CPU times: user 17.3 s, sys: 3.71 s, total: 21 s
Wall time: 7.41 s


In [11]:
prompt = "Tell me about AI"
prompt_template=f'''[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt}[/INST]

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# 17.5s 8856 MiB VRAM



*** Generate:


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<s> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Tell me about AI[/INST]

Hello! I'd be happy to help you learn about AI. Artificial Intelligence (AI) is a field of computer science that focuses on creating intelligent machines that can perform tasks that typically require human intelligence, such as understanding natural language, recognizing images, making decisions, and solving problems. AI has many applications in various industries, including healthcare, finance, transportation, and education.

There ar

In [12]:
print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

# 16.2s 8856 MiB VRAM

*** Pipeline:
[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Tell me about AI[/INST]

Hello! I'd be happy to help answer your questions about AI! Artificial Intelligence (AI) is a rapidly growing field that involves the development of intelligent machines that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. There are many different types of AI, including:

1. Narrow or weak AI: This type of AI is designed to perform a specific task, such as f